## 数据处理
### 数组的操作
1. 创建数组
    1. 形状 类型 值
2. 访问数组
    1. [row,col] 先行后列，从0开始
    2. `:` 表示一个范围，前闭后开：1:3表示1，2两行/列；单独的`:`表示改集合内的所有元素（一整行或一整列等）
    3. `::`表示均匀间隔：`0::3/::3`：表示`0 3 6 9 ...`
    4. 起始偏移量: 终止偏移量: 步长
3. 张量的运算
    1. 运算中的`axis`参数
        1. `axis`参数将规定对指定的某个轴做该运算，如`a.sum(axis=0)`，将对张量的0轴上的每个元素做求和运算，若`a.size = (3,4,5)`，则求和过后的`a.size = (4,5)`。很显然轴0上的数被”合成“一个数了
    2. 按元素的乘法->哈达玛积 Hadamard product
    3. 点积，不同于数学概念上的点积->只是：向量 $\cdot$ 向量，注意在torch中点积和以下两种运算所调用的函数是不同的，分别有严格的类型要求
        1. 矩阵-向量积 `torch.mv()`
        2. 矩阵-矩阵积 `torch.mm()`
    4. 范数 norm -> 是针对向量的概念
        1. $L_1$范数，元素绝对值和
        2. $L_2$范数，平方和开根号
        3. 弗罗贝尼乌斯范数：矩阵元素平方和的平方根

    **任何按元素的一元运算都不会改变其操作数的形状。同样，给定具有相同形状的任意两个张量，任何按元素二元运算的结果都将是相同形状的张量**

4. 微分，梯度
    1. 梯度是函数($f$)对每个自变量($x_n$)求偏导而构成的$1\times n$的行向量,**需要注意的是，列向量的梯度是行向量**;如

    $$\overrightarrow{\nabla}f(\overrightarrow X) = [\frac{\partial{f}}{\partial{x_1}}\quad ...\quad \frac{\partial{f}}{\partial{x_n}}]$$

    2. 雅可比矩阵 Jacobian-matrix, 假设是一个从n维欧氏空间映射到到m维欧氏空间的函数。这个函数由m个实函数组成。这些函数的偏导数(如果存在)可以组成一个m行n列的矩阵，这个矩阵就是所谓的雅可比矩阵
5. 自动求导（梯度）

    1. **梯度**是针对$F:R^n \to R$函数的概念
    2. $y = f(\overrightarrow{x})$其中$y$为标量，则`y.backward()`之后`x.grad`将给出$\overrightarrow x$的每一个于$y$的偏导数
    3. $\overrightarrow y = f(\overrightarrow x)$其中$\overrightarrow y$为向量（一阶张量）则`x.grad`将无法给出一个“偏导数向量”，
   因为torch求微分采用的是`jecobian-matrix`给出一个雅可比矩阵（大小为$n \times m$，其中$n$为$\overrightarrow y$的元素数量，$\overrightarrow m$为x的元素数量）
   这时候需要给`backward()`传递一个`gradient`矩阵，大小为$1 \times n$，让这个向量和雅可比矩阵点乘之后给出最后的梯度结果。
    4. $Y$为二阶张量的情况：...
    5. 当调用`y.backward()`函数之后，整个计算图的内存将被释放，如果再次调用同一个计算图的`backward()`，将报错

In [29]:
import numpy as np
import torch

x = torch.arange(12) # 生成一维的顺序序列
x = x.reshape((3,4)) # 只改变形状

z = torch.zeros((2,3,5))
# z = torch.ones()

# 还可以使用python的List来生成一个tensor

# 按元素进行四则运算和幂运算

# tensor的连接
x = torch.arange(12,dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[100,100,100,100]])
# error: torch.tensor([100,100,100,100]) shape不同无法合并

torch.cat((x,Y),dim=0) # 沿着第0维合并

# 降低维度求和运算
x = torch.arange(24).reshape((2,3,4))
# print(x)
y = x.sum(axis=0)
z = x.sum(axis=1)
j = x.sum(axis=2)
print(x)
print(y)
print(z)
print(j)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])
tensor([[12, 14, 16, 18],
        [20, 22, 24, 26],
        [28, 30, 32, 34]])
tensor([[12, 15, 18, 21],
        [48, 51, 54, 57]])
tensor([[ 6, 22, 38],
        [54, 70, 86]])


In [ ]:
# 降低维度求平均运算
# m = x.mean(axis=0)
# 不降低维度
y = x.sum(axis=0,keepdim=True) # x.cumsum(axis=0)
# print(y)
to_one = x/y #归一化
# print(to_one)

# product
# dotproduct
a = torch.arange(16,dtype=torch.long).reshape((4,4))
b = torch.from_numpy(np.array([1,2,3,4],dtype=np.longlong)).reshape((4,1))# 列方向是向量的默认方向，化为这个shape之后就可以进行 mm 点积运算了
# 矩阵-向量之间不能执行dot运算
# c = a.dot(b)
# RuntimeError: 1D tensors expected, but got 2D and 1D tensors
a.dtype,b.dtype
# c = a.mv(b)
c = a.mm(b)
# RuntimeError: mat2 must be a matrix

In [ ]:
a = torch.arange(3).reshape((3,1))
b = torch.arange(2).reshape((1,2))
# 广播机制需要张量的维度相同
## 都按照每个维度最大量进行复制变换
## 1,3 + 2,1 => 2,3 + 2,3

In [23]:
# -1 表示最后一个元素， -2表示倒数第二个元素...

# 地址的变化
# id(Y) 取Y的地址 类似C的指针

before = id(Y)
Y = Y + x
id(Y) == before
# 注意 Y 的地址已经发生了变化， 相当于 Y+x的时候生成了一个新的tensor，将这个新的tensor赋给了Y

x = torch.arange(12,dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[100.0,100.0,100.0,100.0],[100.0,100.0,100.0,100.0],[100.0,100.0,100.0,100.0]])
before = id(Y)
Y += x
result = (id(Y) == before)
print(result)
# Y的地址不会发生变化 类似于拷贝操作,但是必须保证type和shape一样

z = torch.zeros_like(Y)
print(id(z))
z[:] = Y
print(id(z))
# z的地址不会发生变化 类似于拷贝操作

a = torch.arange(12)
a.reshape((3,4)) # reshape不会返回一个新的tensor，而是在原来的tensor里面进行更改

True
2374120520576
2374120520576


tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [ ]:
# 使用csv文件 逗号分隔值文件
import os
# FiledName1,FiledName2,FiledName3
# value1,value2,value3
# value4,value5,value6
# value7,value8,value9
# ...
# 顶顶顶顶

In [104]:
import torch
# 自动求导
x = torch.ones(3,dtype=torch.float32,requires_grad=True)
y = (x**2)[:2]
print(y)
y.backward(gradient=torch.Tensor([1,1]))
print(x.grad)
# y.backward(gradient=torch.Tensor([1,1]))
# print(x.grad)
# print(x.grad)

tensor([1., 1.], grad_fn=<SliceBackward>)
tensor([2., 2., 0.])


In [147]:
import math
import torch
x = torch.Tensor([1.,2.0])
x.requires_grad_(True)
y = torch.sin(x)
y.sum().backward()
print(x.grad)
print(x.grad == torch.cos(x))

tensor([ 0.5403, -0.4161])
tensor([True, True])
